In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

In [2]:
n = 100
x = np.arange(n)
y = np.abs(np.random.randn(n))
y_cs = np.cumsum(y)

data = pd.DataFrame({"x": x, "y": y, "A": y_cs, "B": 2 * y_cs, "C": 3 * y_cs})

In [28]:
def plot_trend(
    data, xcol, columns_included, column_highlight, shade_l, shade_r
):
    # subplots
    fig = make_subplots(
        rows=1,
        cols=2,
        column_widths=[3, 1],
        row_heights=[1],
    )

    # plot lines in the main plot
    for column in columns_included:
        if column == column_highlight:
            fig.add_trace(
                go.Scatter(
                    x=data[xcol],
                    y=data[column],
                    mode="lines",
                    line=dict(color="rgba(65,40,200,1)"),
                    fill="tozeroy",
                    fillgradient=dict(
                        type="horizontal",
                        colorscale=[
                            (0.0, "darkblue"),
                            (0.5, "royalblue"),
                            (1.0, "cyan"),
                        ],
                    ),
                    showlegend=False,
                ),
                row=1,
                col=1,
            )

        else:
            fig.add_trace(
                go.Scatter(
                    x=data[xcol],
                    y=data[column],
                    mode="lines",
                    line=dict(color="rgba(60,60,60,0.25)"),
                    showlegend=False,
                ),
                row=1,
                col=1,
            )

    # add shaded area
    x_min = min(data[xcol])
    x_max = max(data[xcol])

    def add_rect(fig, l, r, x_min=x_min, x_max=x_max):
        fig.add_vrect(
            x0=x_min,
            x1=l,
            fillcolor="black",
            opacity=0.2,
            line_width=0,
            row=1,
            col=1,
        )
        fig.add_vrect(
            x0=r,
            x1=x_max,
            fillcolor="black",
            opacity=0.2,
            line_width=0,
            row=1,
            col=1,
        )

    add_rect(fig, shade_l, shade_r)

    # get the total of the highlighted area
    def get_y_at_x(x, x_data, y_data):
        return np.interp(x, x_data, y_data)

    y_r = get_y_at_x(shade_r, data["x"], data[column_highlight])
    y_l = get_y_at_x(shade_l, data["x"], data[column_highlight])

    shaded_sum = y_r - y_l

    # plot the bar chart in the second plot
    # first, from 0 to y_l
    fig.add_trace(
        go.Bar(
            x=[0],
            y=[y_l],
            marker=dict(color="rgba(65,40,200,1)"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )
    # then from y_l to y_r
    fig.add_trace(
        go.Bar(
            x=[0],
            y=[shaded_sum],
            marker=dict(color="rgba(65,40,200,0.5)"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )

    # then from y_r to the y(x_max)
    fig.add_trace(
        go.Bar(
            x=[0],
            y=[y_r],
            marker=dict(color="rgba(65,40,200,0.25)"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )

    fig.update_layout(
        font=dict(size=12),
        width=800,
        height=600,
        title=f"Shaded Area: {shaded_sum:.2f}",
    )

    fig.show()

In [27]:
plot_trend(data, "x", ["A", "B", "C"], "B", 20, 40)